In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
np.random.seed(42)

In [ ]:
import os
if os.path.exists('/mnt/data/People.csv'):
    df = pd.read_csv('/mnt/data/People.csv')
else:
    n = 5000
    departments = ['sales','technical','support','IT','product_mng','marketing','RandD','accounting','hr','management']
    salary_levels = ['low','medium','high']
    df = pd.DataFrame({
        'satisfaction_level': np.round(np.clip(np.random.beta(2,5,size=n),0,1),3),
        'last_evaluation': np.round(np.clip(np.random.beta(2,2,size=n),0,1),3),
        'number_project': np.random.randint(2,8,size=n),
        'average_montly_hours': np.random.randint(80,310,size=n),
        'time_spend_company': np.random.randint(1,11,size=n),
        'Work_accident': np.random.choice([0,1],size=n,p=[0.9,0.1]),
        'promotion_last_5years': np.random.choice([0,1],size=n,p=[0.95,0.05]),
        'department': np.random.choice(departments,size=n),
        'salary': np.random.choice(salary_levels,size=n,p=[0.6,0.3,0.1])
    })
    prob_left = (1-df['satisfaction_level']) * 0.6 + (df['time_spend_company']<3)*0.1 + (df['salary']=='low')*0.15 + np.random.rand(n)*0.1
    df['left'] = (prob_left > 0.4).astype(int)
    df.to_csv('/mnt/data/People_synthetic.csv', index=False)

In [ ]:
displayed = df.head(10)
displayed

In [ ]:
print(df.isnull().sum())

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(df[numeric_cols].describe().T)
print(df[numeric_cols].median())

In [ ]:
le = LabelEncoder()
df['salary_level'] = le.fit_transform(df['salary'])
dep_dept = df.groupby('department')['salary_level'].mean().sort_values(ascending=False)
print(dep_dept)
print(dep_dept.idxmax())
print(dep_dept.idxmin())

In [ ]:
corr = df[numeric_cols + ['salary_level','left']].corr()
plt.figure(figsize=(8,6))
plt.imshow(corr, interpolation='nearest', aspect='auto')
plt.colorbar()
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.index)), corr.index)
plt.title('correlation heatmap')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.bar(['Stayed','Left'], df['left'].value_counts().sort_index().values)
plt.title('Stayed vs Left')
plt.show()

In [ ]:
print(df.groupby('department')['left'].mean().sort_values(ascending=False).head())

In [ ]:
X = df[['satisfaction_level','last_evaluation','number_project','average_montly_hours','time_spend_company','Work_accident','promotion_last_5years','salary_level']]
X = pd.concat([X, pd.get_dummies(df['department'], prefix='dept')], axis=1)
y = df['left']
scaler = StandardScaler()
num_cols = ['satisfaction_level','last_evaluation','number_project','average_montly_hours','time_spend_company']
X[num_cols] = scaler.fit_transform(X[num_cols])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [ ]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('accuracy', accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('rf accuracy', accuracy_score(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

In [ ]:
plt.figure()
df.groupby('salary')['left'].mean().plot(kind='bar')
plt.title('Attrition rate by salary level')
plt.show()

In [ ]:
plt.figure()
groups = df.groupby('salary')['time_spend_company']
plt.boxplot([groups.get_group(g).values for g in groups.groups], labels=list(groups.groups.keys()))
plt.title('Time in company by salary')
plt.show()

In [ ]:
plt.figure()
plt.scatter(df['average_montly_hours'], df['satisfaction_level'], alpha=0.4)
plt.xlabel('average_montly_hours')
plt.ylabel('satisfaction_level')
plt.title('Hours vs Satisfaction')
plt.show()

In [ ]:
plt.figure()
df.groupby('department')['left'].sum().plot(kind='pie', autopct='%1.1f%%')
plt.title('Attrition count by department')
plt.ylabel('')
plt.show()

In [ ]:
sql = [
"SELECT department, AVG(CASE WHEN salary='low' THEN 1 WHEN salary='medium' THEN 2 WHEN salary='high' THEN 3 END) AS avg_salary_numeric FROM employees GROUP BY department ORDER BY avg_salary_numeric DESC LIMIT 1;",
"SELECT * FROM employees WHERE years_at_company > 5 AND promotion_last_5years = 0;",
"SELECT department, SUM(CASE WHEN left=1 THEN 1 ELSE 0 END)*1.0/COUNT(*) AS attrition_rate FROM employees GROUP BY department ORDER BY attrition_rate DESC LIMIT 3;",
"SELECT AVG(satisfaction_level) FROM employees WHERE left = 1;",
"SELECT department FROM employees GROUP BY department ORDER BY AVG(average_montly_hours) ASC LIMIT 1;"
]
with open('/mnt/data/sql_queries.sql','w') as f:
    for q in sql:
        f.write(q + '\n')